In [1]:
pathtorepo = "C:\\Users\\Alex\\Desktop\\"

using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")

using StaticArrays, DifferentialEquations, DynamicalSystems

x, y, z  = -10..10, -10..10,  -10..10
box = x × y × z

using CairoMakie
using GLMakie
using ForwardDiff
using LinearAlgebra

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


In [2]:
@inbounds function Res(u, p, t)
    du1 = -u[2]-u[3]
    du2 = u[1]+p[1]*u[2]
    du3 = p[2]*u[1]-p[3]*u[3]+u[1]*u[3]
    return SVector(du1, du2, du3)
end
@inbounds function jac_res(u, p, t)
    SMatrix{3, 3}(0.0, 1.0, p[2]+u[3],
                -1.0, p[1], 0.0,
                -1.0, 0.0, -p[3]+u[1])
end

jac_res (generic function with 1 method)

In [10]:
time_set = (t = 500, Ttr = 250)
integ_set = (alg = Vern9(), abstol=1e-10, reltol=1e-10)

(alg = Vern9(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false), lazy = true), abstol = 1.0e-10, reltol = 1.0e-10)

In [11]:
 b = 0.3;  c = 4.89; a = 0.35;

In [12]:
u0 = SA[0.13060105886751308, 0.4782615699018184, 0.7435866656901606]
p = [a, b, c]
ds = CoupledODEs(Res, u0, p, diffeq = integ_set)

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  Res
 ODE solver:    Vern9
 ODE kwargs:    (abstol = 1.0e-10, reltol = 1.0e-10)
 parameters:    [0.35, 0.3, 4.89]
 time:          0.0
 state:         [0.13060105886751308, 0.4782615699018184, 0.7435866656901606]


In [13]:
fp, ei, _ = fixedpoints(ds, box, jac_res)

(3-dimensional StateSpaceSet{Float64} with 1 points, Vector{ComplexF64}[[-4.8302785210864085 + 0.0im, 0.1451392605432047 - 0.9846627344351879im, 0.1451392605432047 + 0.9846627344351879im]], Bool[0])

In [47]:
fixedpoint = SA[0.0, 0.0, 0.0]

Jacobian = jac_res(fixedpoint, p, 0)
eivecs = eigvecs(Jacobian)

vec_stable = real(eivecs[:, 1])
ϵ = 1e-23

shift = fixedpoint + vec_stable*ϵ

3-element SVector{3, Float64} with indices SOneTo(3):
  1.9510202110332117e-24
 -3.7662457782753524e-25
  9.800595597386729e-24

In [56]:
prob = ODEProblem(Res, shift, (0.0, 1000.0),p)

ODEProblem with uType SVector{3, Float64} and tType Float64. In-place: false
timespan: (0.0, 1000.0)
u0: 3-element SVector{3, Float64} with indices SOneTo(3):
  1.9510202110332117e-24
 -3.7662457782753524e-25
  9.800595597386729e-24

In [64]:
sol = solve(prob,alg = Feagin14(), abstol=1e-12, reltol=1e-12, maxiters = 10000000)

retcode: Success
Interpolation: 3rd order Hermite
t: 10110-element Vector{Float64}:
    0.0
    9.999999999999999e-5
    0.0010999999999999998
    0.011099999999999997
    0.11109999999999996
    0.817205675759913
    1.6451597957206654
    2.030192686727324
    2.368994626272536
    2.7132460059643835
    3.0814637340683477
    3.4919085529180887
    3.9121455101813036
    ⋮
  999.153989010118
  999.2133740965569
  999.2746654752711
  999.3382205528827
  999.4045568459057
  999.4743372664436
  999.5486621742677
  999.6292202702116
  999.718974410103
  999.8248589058873
  999.9612248285761
 1000.0
u: 10110-element Vector{SVector{3, Float64}}:
 [1.9510202110332117e-24, -3.7662457782753524e-25, 9.800595597386729e-24]
 [1.9500780414966383e-24, -3.764427015958349e-25, 9.795862779879295e-24]
 [1.940681334045444e-24, -3.7462876294119595e-25, 9.74866012705422e-24]
 [1.8491689636413493e-24, -3.569632320181786e-25, 9.2889643589546e-24]
 [1.1407757082372348e-24, -2.202151300540468e-25, 5.7304795

In [66]:
ts, tf = 1, 10000
idx, idy, idz = 1, 2, 3
f = Figure(resolution = (1200, 800))
ax = Axis3(f[1, 1])
lines!(ax, sol[idx, ts:tf], sol[idy, ts:tf], sol[idz, ts:tf], linewidth = 1.0, color = :deeppink)
#scatter!(fp[1][idx], fp[1][idy], fp[1][idz], color = :black)
display(GLMakie.Screen(), f)

GLMakie.Screen(...)